In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Embedding, LSTM
from keras.utils import np_utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
def loadData():
    with open('dataset.csv', encoding='utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        data = [row for row in readCSV]
        return (data)
    
dataset = loadData()
max_length = max([len(row[0]) for row in dataset])
print('data length =', len(dataset))
print('max str length =', max_length)

data length = 63774
max str length = 28


In [3]:
def name2vec():
    X_train = []
    for row in dataset:
        X_train.append(row[0])
    return np.array(X_train)

X_train = name2vec()

In [4]:
tokenizer = Tokenizer(num_words=5000, lower=True,split=' ')
tokenizer.fit_on_texts(X_train)
nb_words = len(tokenizer.word_index.keys()) + 1
X = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(X)

In [5]:
def getGenders():
    Y_train = []
    for row in dataset:
        if (row[1] == 'm'):
            Y_train.append([1.0, 0.0])
        else:
            Y_train.append([0.0, 1.0])
    return np.array(Y_train)

Y = getGenders()

In [6]:
embed_dim = 128
lstm_out = 200

model = Sequential()
model.add(Embedding(nb_words, embed_dim, input_length=X.shape[1]))
model.add(Dropout(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 6, 128)            388352    
_________________________________________________________________
dropout (Dropout)            (None, 6, 128)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 200)               263200    
_________________________________________________________________
dense (Dense)                (None, 2)                 402       
Total params: 651,954
Trainable params: 651,954
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(X, Y, batch_size=32, epochs=20, verbose=1)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
63774/63774 [==============================] - 32s 494us/step - loss: 0.1880 - acc: 0.9176
Epoch 2/20
63774/63774 [==============================] - 29s 462us/step - loss: 0.1106 - acc: 0.9568
Epoch 3/20
63774/63774 [==============================] - 29s 455us/step - loss: 0.0991 - acc: 0.9610
Epoch 4/20
63774/63774 [==============================] - 29s 455us/step - loss: 0.0929 - acc: 0.9635
Epoch 5/20
63774/63774 [==============================] - 29s 452us/step - loss: 0.0883 - acc: 0.9662
Epoch 6/20
63774/63774 [==============================] - 30s 464us/step - loss: 0.0845 - acc: 0.9670
Epoch 7/20
63774/63774 [==============================] - 29s 454us/step - loss: 0.0824 - acc: 0.9681
Epoch 8/20
63774/63774 [==============================] - 29s 453us/step - loss: 0.0798 - acc: 0.9693
Epoch 9/20
63774/63774 [==============================] - 29s 455us/step - loss: 0.0788 - acc: 0.9690
Epoch 10/20
63774/63774 [==============================] - 29s 455us/step - loss: 

In [8]:
def gender(name):
    inp = tokenizer.texts_to_sequences(np.array([name]))
    inp = pad_sequences(inp, maxlen=6)
    pred = model.predict(inp)[0]
    if pred[0] > pred[1]:
        return "nam"
    else:
        return "nữ"

gender("kiều thùy trang")

'nữ'

In [9]:
model.save_weights('./model.h5')

Đoạn này vì không thể cài `tensorflowjs` trên env hiện tại, nên mình phải save model lại và chuyển sang env khác để export model ra json

In [12]:
import tensorflowjs as tfjs
model.load_weights('./model.h5')
tfjs.converters.save_keras_model(model, './modeljs')